# Reading WSIS database

Datasets:
- `wosis_201909_attributes.tsv`: This file lists the four to six letter code for each attribute,
whether the attribute is a site or horizon property, the unit of measurement, the number of
profiles respectively layers represented in the snapshot, and a brief description of each
attribute, as well as the inferred uncertainty for each property
- `wosis_201909_profiles.tsv`: This file contains the unique profile ID (i.e. primary key), the
source of the data, country ISO code and name, accuracy of geographical coordinates,
latitude and longitude (WGS 1984), point geometry of the location of the profile, maximum
depth of soil described and sampled, as well as information on the soil classification system
and edition. Depending on the soil classification system used, the number of fields will vary. 
- `wosis_201909_layer_chemical.tsv` and `wosis_201909_layer_physical.tsv`: The layer
(horizon) data are presented in two separate file in view of their size, one for the chemical
and one for the physical soil properties.

In [ ]:
import geopandas
import pandas as pd
import matplotlib.pyplot as plt

The `wosis_201909.gpkg` file is the same as the `wosis_201909_profiles.tsv` file + `geometry` column.

In [ ]:
gdf = geopandas.read_file("../data/WoSIS_2019_September/wosis_201909.gpkg")

In [ ]:
gdf.sample(5)

In [ ]:
attributes = pd.read_csv('../data/WoSIS_2019_September/wosis_201909_attributes.tsv', sep='\t')
# profiles = pd.read_csv('../data/WoSIS_2019_September/wosis_201909_profiles.tsv', sep='\t')
layers_chemical = pd.read_csv('../data/WoSIS_2019_September/wosis_201909_layers_chemical.tsv', sep='\t')
layers_physical = pd.read_csv('../data/WoSIS_2019_September/wosis_201909_layers_physical.tsv', sep='\t')

In [ ]:
gdf.columns

In [ ]:
gdf["geometry"].plot()

Attributes:

In [ ]:
attributes.sample(5)

**ORGC** is the code for organic carbon:

In [ ]:
attributes.iloc[23]

**ORGC** is a chemical property, thus can be seen in `layers_chemical`:

In [ ]:
layers_chemical.orgc_value

Save carbon observations:

In [ ]:
carbon = layers_chemical[layers_chemical['orgc_value'].notna()]

Then, it is useful to keep only the **relevant columns** from `layers_chemicals`:

Save the codes to drop:

In [ ]:
attribute_codes = attributes.code.drop(23).tolist()

and columns associated to each attribute:

In [ ]:
attribute_columns = ["value", "value_avg", "method", "date",
                    "dataset_id", "profile_code", "licence"]

In [ ]:
[f"{code.lower()}_{column}" for column in attribute_columns]

In [ ]:
columns_eliminated = 0
for code in attribute_codes:
    try:
        carbon.drop([f"{code.lower()}_{column}" for column in attribute_columns], axis=1, inplace=True)
        columns_eliminated += 1
    except:
        pass
print(f"{columns_eliminated} have been eliminated")

This dataset allows us to zoom on carbon:

In [ ]:
carbon.columns

In [ ]:
carbon.sample(5)

In [ ]:
carbon.profile_id

I notice how `gdf` does not have a `profile_id` column. Therefore, I derive it from the profiles file.

In [ ]:
profiles = pd.read_csv('../data/WoSIS_2019_September/wosis_201909_profiles.tsv', sep='\t')

In [ ]:
gdf["profile_id"] = profiles.profile_id.copy()

Select only profiles with carbon observations:

In [ ]:
carbon_profiles = gdf.loc[gdf.profile_id.isin(carbon.profile_id.unique())].copy()

There are 35259 different profiles in the US. It might make sense to zoom into US fist.

In [ ]:
carbon_profiles[["country_name", "profile_id"]].groupby("country_name").agg("count").sort_values("profile_id", ascending=False)

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))  # base map

In [ ]:
base = world.plot(figsize=(15,10), color='white', edgecolor='black')

carbon_profiles["geometry"].plot(ax=base, marker='o', color='red', markersize=0.01)

Looking at the US more in details:

In [ ]:
US = world.query('name == "United States of America"')
base = US.plot(figsize=(10,10), color='white', edgecolor='black')

carbon_profiles.loc[carbon_profiles["country_name"] == "United States of America"]["geometry"].plot(ax=base, marker='o', color='red', markersize=0.01)